## Генерация текста с помощью модели **GPT**

https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 49.8 MB/s 
     |████████████████████████████████| 163 kB 65.2 MB/s 


In [3]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#токенайзер модели
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [9]:
#файл с текстом, на котором дообучаем модель
import re
with open('/content/drive/MyDrive/pelevin.txt') as f:
    text = f.read()

text = re.sub('\n{2,}', '\n', text)
print(text[:1000])

Омон Ра
роман
Героям Советского Космоса
Омон – имя не особо частое и, может, не самое лучшее, какое бывает. Меня так назвал отец, который всю свою жизнь проработал в милиции и хотел, чтобы я тоже стал милиционером.
– Пойми, Омка, – часто говорил он мне, выпив, – пойдешь в милицию – так с таким именем, да еще если в партию вступишь…
Хоть отцу и приходилось иногда стрелять в людей, он был человек незлой души, по природе веселый и отзывчивый. Меня он очень любил и надеялся, что хотя бы мне удастся то, что не удалось в жизни ему. А хотел он получить участок земли под Москвой и начать выращивать на нем свеклу и огурцы – не для того, чтобы продавать их на рынке или съедать, хотя и это все тоже, а для того, чтобы, раздевшись до пояса, рубить лопатой землю, смотреть, как шевелятся красные черви и другая подземная жизнь, чтобы возить через весь дачный поселок тачки с навозом, останавливаясь у чужих калиток побалагурить. Когда он понял, что ничего из этого у него не выйдет, он стал надеяться, чт

In [10]:
# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

165361


array([ 630, 2810, 2086,  203,  620,  363,  203, 1009, 1960, 2078])

In [11]:
# разбиваем на train и test

train = []
test = []
new_lines = np.where(tokens == tokenizer.encode('\n'))[0]
i = 0
train.extend(tokens[0: new_lines[0]])
while i < (len(new_lines)-20):
    train.extend(tokens[new_lines[i]: new_lines[i+18]])
    test.extend(tokens[new_lines[i+18]: new_lines[i+20]])
    i+=20
train.extend(tokens[new_lines[i]: new_lines[-1]])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

165361 150547 14813


In [12]:
from transformers import GPT2LMHeadModel

# Эту модель просто подгружаем и не будем дообучать 
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [13]:
batch_size = 8
max_len = 256
epochs = 5

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


# зададим точность, хотя ориентироваться будем на качество генерации
def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

73 7


In [14]:
torch.cuda.empty_cache()

In [15]:
# готовим тензоры для обучения размера [batch_size, max_len]

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/5 : training


  0%|          | 0/73 [00:00<?, ?it/s]

epoch 1/5 : validation


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 2/5 : training


...

  0%|          | 0/7 [00:00<?, ?it/s]

epoch 5/5 : training


  0%|          | 0/73 [00:00<?, ?it/s]

epoch 5/5 : validation


  0%|          | 0/7 [00:00<?, ?it/s]

In [16]:
import textwrap

In [20]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# модель без дообучения

# prompt = "print('Hello world!') "
prompt = 'Я начал'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    # входная строка
    input_ids=prompt,
    # максимальная длина генерируемой последовательности
    max_length=150,
    # num_beams
    num_beams=5,
    # применяем сэмплирование
    do_sample=True,
    # применяем температуру
    temperature=100.,
    # топ слов по вероятности
    top_k=50,
    # топ слов по суммарной вероятности
    top_p=0.6,
    # сколько (постараться) не повторять n_gram подряд
    no_repeat_ngram_size=3,
    # сколько вернуть генераций
    num_return_sequences=7,
    ).cpu().numpy()

# out содержит результаты

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 100), end='\n------------------\n')

Я начал свой проект «Рукодельничьи радости». И если на меня с первых слов было не очень радостно (но
зато приятно), когда мне показали мое любимое место в магазине. Мне было комфортно – так это была
моя комната:) И тут в ней, по всему было написано мое самое важное «а чего еще я не делала. Ну да не
буду, что там еще есть:) Вот что бы сделать с моим ребенком – сделать очень хорошо. Это же так
прекрасно! Так приятно и замечательно делать такое с ребенком. Но все это мелочи – дети должны
заниматься. Мне приятно было делать их своими. В итоге это были наши общие детские мечты в жизни – и
мечты эти тоже очень интересные. Но в них была моя роль: делать, как ребенок может,
------------------
Я начал искать людей по имени Джуджани в Америке или других районах мира К. Биглер-Мейн, Майкл С.,
Оуэн О., Дэвид Дюкью-Р. Дукалси-Эснер; Лоутон Бэклинсон - Джон, Джозетта Маккинг-Хэй и Сир. Д.
Рибейрд-Вир. Кэннер-Гэйб-Сентер, Джеймс Т., Раген Т. В., Кэндлер Пью-Хай, Эдер Маккленс; Чилл
Маккинкэ - Ривелл-

In [31]:
# дообученная модель
prompt = 'Я начал'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=1,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    a = tokenizer.decode(out_).rfind('.')
    print(textwrap.fill(tokenizer.decode(out_)[0:tokenizer.decode(out_).rfind('.')+1], 100), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Я начал думать о том, что, если бы я был человеком, я бы, наверное, никогда не стал таким, как
сейчас, – подумал я. – Ну и что? – спросил кто-то из нас. – Ты думаешь, это так просто? Я посмотрел
на часы. Было уже около половины десятого, когда я закончил думать. Я встал, подошел к окну и
выглянул на улицу. На улице было совсем темно, и только в нескольких окнах горел свет, но я не мог
понять, где я нахожусь, потому что вокруг было так тихо и спокойно, словно я находился в каком-
нибудь другом мире, а не в своем собственном. Потом я вспомнил, кто я такой и почему я здесь.
------------------
Я начал думать о том, что, если бы я был жив, я бы, наверное, умер. Но я не знал, как это сделать. –
А я знаю, – сказал я. – Ты знаешь, кто я такой? Он посмотрел на меня, и я увидел в его глазах что-то
такое, от чего мне стало не по себе. Потом он поднял голову и посмотрел мне в глаза. Я понял, о чем
он говорит, но ничего не сказал. Тогда он встал и подошел ко мне. Он взял меня за руку и отвел в
стор

In [24]:
torch.save(model.state_dict(), '/content/drive/MyDrive/pelevin.pt')